# Welcome to "Create Your Own Writer Algorithm"!
Or a text generator, as you would usually call it ...

![Den Norske Dataforeningen](imgs\dnd.png "Title")
![Elkjøp Nordic](imgs\elkjop.jpg "Title")

- *Content by Kjetil Åmdal-Sævik*
- *Code by Keras GitHub repository*
- *Pizza by Den Norske Dataforeningen*
- *Computing power by Elkjøp Nordic*

What we're going to do today is give some text to an algorithm and teach it to write something similar, character by character. If you have your own text that you'd like to use, put it in the "data"-folder and remember the filename. 

For this, we're going to use something called a recurrent neural network – an RNN. "Regular" neural networks take a set of inputs and apply a series non-linear transformations to predict an output. In our case, that could be 1) trying to predict the next letter in a sequence based on only the previous letter as input, or 2) maybe based on multiple previous letters as inputs but without explicitly knowing their order.
<br><br><br>
<center>
#### "Regular" Deep Neural Network
</center>
![Deep Neural Network](imgs\dnn.png "Title")
<br><br>
Let's say you have the sentence **hello worl** and want to predict the next letter. 

The right answer here is obviously **d**. However:
- With 1), only using the one previous letter, you might as well predict **o** or **l**, since these two characters can also follow an **l**.
- With 2), using all the letter but not knowing their order, the algorithm might as well think the sentence is **hell or low** (let's say there's a space before "hello" that's part of the sequence) and predict a space as the next character.

There is of course a 3) here; you can explicitly create a different feature set for each part of the sequence, so that having an **l** as the previous letter is a different feature than having an **l** as the letter before that. But this is terribly inefficient, since the algorithm will still need to learn that there is a sequential aspect to these features.

RNNs don't have any of these problems, because the sequential aspect is built into their architecture by definition. RNNs have mechanisms that understand that the letters **hello worl** occur in that exact order if we feed the data correctly.
<br><br><br>
<center>
#### Recurrent Neural Network
</center>
![Recurrent Neural Network](imgs\rnn.png "Title")
<br><br>
More specifically, the RNN we're going to train is an LSTM network. LSTM stands for Long Short-Term Memory, and is basically a "better" RNN for learning long-term dependencies. There is nothing basic about the mathematics behind it, but you usually don't really need to know any of that to train good models!

*Please note that there are multiple ways to implement RNNs. The Keras implementation is actually quite strange if you're familiar with how RNNs usually work, and it might be considered "inelegant". However, it works quite well and it's easier to understand than many other implementations.*

### Import required modules
As mentioned, we're going to use Keras with the TensorFlow (default) backend. This means we have to import the Keras library, or better yet; just the things we need from the Keras library. We also import NumPy to create arrays to feed the data into our model and some helper functions.

In [ ]:
from keras.models import Sequential # This is the Keras model class itself
from keras.layers import Dense, LSTM # These are two types of layers for our networks; the recurrent layer and the output layer
from keras.utils.data_utils import get_file # This is just a helper function to download a text file if needed

import numpy as np # Contains the required classes and functions to correctly build the input to our model
import random # Allows us to do random sampling
import sys # Allows us to generate samples while we're training the model

*For this particular session we need to do something special: since we're going to train on the GPU, we have to set a limit for the GPU memory allocated to each of our sessions, since Keras and TensorFlow by default allocates everything to one session. The following code fixes this.*

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
set_session(tf.Session(config=config))

### Import training data
First, we have to get a text file containing data for the algorithm to train on. 

You can decide if you want to use the writings of Nietzsche, or all of rosablogger Sophie Elise Isachsens blog posts from 2015. Just uncomment the line referring to whicever text you want to use!

In [ ]:
# Download text file containing the writings of Nietzsche, or read Sophie ELise Isachsens blog posts from 2015
#path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
path = 'data/sophie_elise_text.txt'

# Open and read file, and convert to lower case
text = open(path, encoding='utf-8').read().lower()

# Print the number of characters in the text
print('corpus length:', len(text))

### Create a dictionary of characters
Neural networks can't really read text – they only deal with numbers. Therefore we need to create some dictionaries to map the characters in our text to numbers and help us convert back and forth between characters and numbers.

In [ ]:
# Get a list of distinct characters and sort it
chars = sorted(list(set(text)))

# Print the number of distinct characters in the text
print('total chars:', len(chars))

# Create a dictionary for mapping characters to indices
char_indices = dict((c, i) for i, c in enumerate(chars))

# Create a dictionary for mapping indices to characters
indices_char = dict((i, c) for i, c in enumerate(chars))

### Put text into arrays
Now that we know which character corresponds to which number and vice versa, we can use this information to convert our text into arrays. This is usually the easiest way to feed data into any neural network library, at least in Python. It works very well for prototyping and for hobby stuff, but there are more efficient ways to do it in a production environment.

We create two arrays: 
- An array of X'es, containing sequences of a certain number of characters that we define ourselves.
- An array of y's, containing the next character following the sequence in the corresponding row in X.

We have already mapped our characters to numbers, but there are two more things that we need to take into account:
1. Neural networks prefer their inputs to be in the 0-1 range. Our characters are integers in the 0-58 range (for Nietzsche) or 0-77 range (for Sophie Elise).
2. Now that we're dealing with numbers instead of text, we have implicitly stated that a < b, b < c and so on. This doesn't really make sense. It makes more sense to treat characters as unordered categorical data.

The first issue is usually solved by min-max scaling. However, that's not really appropriate for categorical data in this format. Therefore, we solve this by doing so-called **one hot encoding**. This basically means we treat each category as a binary feature which can be 0 or 1. 

For Sophie Elise's text, that means we get 78 features per row, where each feature corresponds to one character, and only one of those features is 1 at a given timestep in a sequence. 

In [ ]:
# Cut the text into semi-redundant sequences of length "maxlen", skipping "step" number of characters for each sequence
# First, declare the required parameters, and lists to contain our X's and Y's
maxlen = 30
step = 5
sentences = []
next_chars = []

# Loop through the text, character by character and skipping by "step", and add the elements to lists
# Note that we're still dealing with text here; the number conversion is the next step after this
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('nb sequences:', len(sentences))

# Then, create the arrays to hold the 0's and 1's
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

# Finally, loop through the lists we created before and fill in the one-hot vectors
print('Vectorization...')
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print('Done!')

### Build the model
Now it's time to actually build our text generator model! That must be pretty complex and time-consuming, right?

In [ ]:
# Build a simple RNN in Keras with LSTM cells
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

**What, only five lines??**

Yes indeed! With Keras you can define quite complex models with very few lines of code.

This session is supposed to be very light on the mathematics, so we're not going to go through the inner workings of RNNs. 

Intuitively, you can imagine that RNNs learn the features and properties of sequences in the same way that convolutional neural networks (CNNs, if you're familiar with those) learn the features and properties of images. You don't explicitly tell a CNN that a person consists of a head, a torso, two arms and two legs, and that a head consists of two eyes (among other things), which again consists of multiple sets of circles etc. These are the things the model learns by itself. 

RNNs work in the same way, only for sequential data instead of spatial data. A character-level RNN, such as the one we're going to build, first learns how to spell, and then it learns some form of limited grammar. And with spelling and grammar, our model have learned the concepts of words and sentences. These things are features of sequences of text, in the same way that circles, eyes and heads are features of images. 

Interestingly, this applies to any kind of sequence. For example, good 'ol regular time series data usually consists of trend, seasonality and cycles. It's very likely that if you train an RNN on such data, it will pick up these exact features.

### Define sampling function
Before we start training, we can define a sampling function to help us generate text.

Our model is going to be predicting the probability of the next character, which leaves it up to us to decide what to do with this information. We could just pick the character with the highest probability, but we will get more interesting results by sampling from the distribution and manipulating it a little bit.

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Train the model and see what it learns!
Now it's time to train our model! We do this slightly differently and more "manually" than you would do with other models in Keras, since we want to generate some samples in between and see what the generated text looks like. Usually, training models in Keras is one line of code!

In [ ]:
# Train the model and generate some samples between each iteration
for iteration in range(1, 21):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    
    # The actual training (this is really all you need to train Keras models)
    model.fit(X, y, batch_size=256, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    # This loop prints samples from the algorithm between each training iteration, so we can see how we're doing
    for diversity in [0.4, 0.8, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

### And that's it – you've successfully trained your own writer algorithm!

*Don't forget the biggest happening of the year!*
![Recurrent Neural Network](imgs\mdsa.png "Title")